In [1]:
import re
import json
import pandas as pd
import os
from collections import defaultdict
import random

In [2]:
input_path = "Simulaciones/sim_1_normal.tsv"
df = pd.read_csv(input_path, sep="\t")
simulation_ids = df['simulation_id'].unique()
sim_id = random.choice(simulation_ids)
print(sim_id)

0


### Calcular posiciones de palabras

In [3]:
SCREEN_WIDTH = 1024
SCREEN_HEIGHT = 768
TEXT_LEFT = 150
TEXT_RIGHT = 900
TEXT_TOP = 100
LINE_HEIGHT = 50
LINES_PER_SCREEN = 10
PARAGRAPH_INDENT = 40
WORD_SPACING = 10
LOWERCASE_WIDTH = 15
UPPERCASE_WIDTH = 20

def get_word_width(word):
    width = 0
    if word == "Él":
        return 25 
    if word == "Pero":
        return 55
    if word == "sabía,":
        return 95
    if word == "Pasa":
        return 0
    for c in word:
        if c.isupper():
            if c == "I":
                print(word)
                width += UPPERCASE_WIDTH - 6
            else:
                width += UPPERCASE_WIDTH
        elif c.islower(): 
            if c == 'l':
                width += LOWERCASE_WIDTH - 2
            elif c == 'i':
                width += LOWERCASE_WIDTH - 2
            elif c == 't':
                width += LOWERCASE_WIDTH - 2
            else:
                width += LOWERCASE_WIDTH
        else:
            if c == ")":
                width += LOWERCASE_WIDTH
            elif c == "(":
                width += LOWERCASE_WIDTH
            elif c == "\"":
                print(word)
                print("")
                width += LOWERCASE_WIDTH - 4
            else:    
                width += LOWERCASE_WIDTH + 2
                
                    
    return width


def text_to_screens(text):
    
    # Separar párrafos pero conservar posición de palabras
    lines = []
    for paragraph in text.strip().split("\n"):
        words = re.findall(r"\S+", paragraph)
        #print(words)
        if words:
            lines.append({"words": words, "is_paragraph_start": True})
    
    # Aplanar lista de palabras y marcar si son inicio de párrafo
    flat_words = []
    for line in lines:
        for i, word in enumerate(line["words"]):
            flat_words.append({
                "word": word,
                "is_paragraph_start": line["is_paragraph_start"] and i == 0
            })

    screens = []
    screen_index = 0
    screen_word_counter = 0
    line_index = 0
    y_cursor = TEXT_TOP + (LINE_HEIGHT // 2)
    current_screen = {"screen_index": screen_index, "words": []}
    
    i = 0
    words_read = 0
    while i < len(flat_words):
        # Si llegamos al límite de líneas por screen
        if line_index == LINES_PER_SCREEN:
            screens.append(current_screen)
            screen_index += 1
            screen_word_counter = 0 
            current_screen = {"screen_index": screen_index, "words": []}
            line_index = 0
            y_cursor = TEXT_TOP + (LINE_HEIGHT // 2)
        
        # Armar una línea
        line_words = []
        line_width = 0
        j = i

        while j < len(flat_words):
            word = flat_words[j]["word"]
            #print("WORD:", word)
            word_width = get_word_width(word)
            is_paragraph_start = flat_words[j]["is_paragraph_start"]
            #print(is_paragraph_start)
            #print("word width:", word_width)
            if line_width > 0 and is_paragraph_start:
                break
            if screen_index == 22 and line_index == 0:
                line_width += 5
            # Sangría solo si es la primera palabra de la línea y de un párrafo
            effective_indent = PARAGRAPH_INDENT if len(line_words) == 0 and is_paragraph_start else 0
            total_required = line_width + word_width + effective_indent
            word_required = word_width + effective_indent
            #print("total_required:", total_required)
            if total_required > (TEXT_RIGHT - TEXT_LEFT):
                break

            line_words.append((flat_words[j], word_width, effective_indent))
            line_width += word_required + WORD_SPACING
            j += 1
            words_read += 1

        # Calcular posiciones (x, y)
        x_pos = TEXT_LEFT
        for idx, (word_data, width, indent) in enumerate(line_words):
            if idx == 0:
                x_pos += indent
            word_x_center = x_pos + width / 2
            current_screen["words"].append({
                "word": word_data["word"],
                "x": word_x_center,
                "y": y_cursor,
                "line": line_index,
                "index": idx,
                "screen_word_index": screen_word_counter
            })
            screen_word_counter += 1
            x_pos += width + WORD_SPACING

        line_index += 1
        y_cursor += LINE_HEIGHT
        i = j

    if current_screen["words"]:
        screens.append(current_screen)

    return screens  

def get_screen_text(screens, screen_number):
    if screen_number >= len(screens):
        return f"[ERROR] No existe la screen número {screen_number}."

    screen = screens[screen_number]
    lines = defaultdict(list)

    # Agrupar palabras por línea
    for word_info in screen["words"]:
        line_number = word_info["line"]
        lines[line_number].append((word_info["index"], word_info["word"]))

    # Ordenar palabras y construir el texto
    output_lines = []
    for line_number in sorted(lines.keys()):
        sorted_words = sorted(lines[line_number], key=lambda x: x[0])
        line_text = " ".join(word for _, word in sorted_words)
        output_lines.append(line_text)

    return "\n".join(output_lines)

def build_screen_dataframe(screens):
    data = []
    i = 0
    for screen in screens:
        print(f"\nTexto de la screen {i}:")
        screen_index = screen["screen_index"]
        screen_text = get_screen_text(screens, screen_index)
        print(screen_text)
        data.append({"index": screen_index, "all": screen_text})
        i += 1
    return pd.DataFrame(data)


In [4]:
with open("Axolotl.txt", "r", encoding="utf-8") as f:
    texto = f.read()

In [5]:
len(texto.split())

1975

In [6]:
screens = text_to_screens(texto)
print(f"Se generaron {len(screens)} screens y se guardaron en 'screens.json'")

Iba
"Sálvanos,

sálvanos".

sálvanos".

"Usted

ojos",

Inútilmente
Se generaron 24 screens y se guardaron en 'screens.json'


In [7]:
df_ob1 = build_screen_dataframe(screens)


Texto de la screen 0:
Hubo un tiempo en que yo pensaba mucho en los
axolotl. Iba a verlos al acuario del Jardín des
Plantes y me quedaba horas mirándolos, observando su
inmovilidad, sus oscuros movimientos. Ahora soy un
axolotl.
El azar me llevó hasta ellos una mañana de
primavera en que París abría su cola de pavo real
después de la lenta invernada. Bajé por el bulevar de
Port Royal, tomé St. Marcel y L'Hôpital, vi los
verdes entre tanto gris y me acordé de los leones.

Texto de la screen 1:
Era amigo de los leones y las panteras, pero nunca
había entrado en el húmedo y oscuro edificio de los
acuarios. Dejé mi bicicleta contra las rejas y fui a
ver los tulipanes. Los leones estaban feos y tristes
y mi pantera dormía. Opté por los acuarios, soslayé
peces vulgares hasta dar inesperadamente con los
axolotl. Me quedé una hora mirándolos, y salí incapaz
de otra cosa.
En la biblioteca Saint-Geneviève consulté un
diccionario y supe que los axolotl son formas

Texto de la screen 2:
larvales,

In [8]:
df_ob1

,index,all
0,0,Hubo un tiempo en que yo pensaba mucho en los\...
1,1,"Era amigo de los leones y las panteras, pero n..."
2,2,"larvales, provistas de branquias, de una espec..."
3,3,se usa más) como el de hígado de bacalao.\nNo ...
4,4,uniéndonos. Me había bastado detenerme aquella...
5,5,acuario. Aislé mentalmente una situada a la de...
6,6,en uñas minuciosamente humanas. Y entonces des...
7,7,estatuilla corroída por el tiempo. La boca est...
8,8,"movía apenas, yo veía los diminutos dedos posá..."
9,9,espacio y el tiempo con una inmovilidad indife...


In [9]:
# 1. Reemplazar saltos de línea por espacio
df_ob1["all"] = df_ob1["all"].str.replace("\n", " ", regex=False)

# 2. Eliminar comillas dobles
df_ob1["all"] = df_ob1["all"].str.replace('"', '', regex=False)

In [10]:
df_ob1.to_csv("Axolotl.csv", index=False)

In [11]:
with open("screens.json", "w", encoding="utf-8") as f:
    json.dump(screens, f, ensure_ascii=False, indent=2)

In [12]:
screens

[{'screen_index': 0,
  'words': [{'word': 'Hubo',
    'x': 222.5,
    'y': 125,
    'line': 0,
    'index': 0,
    'screen_word_index': 0},
   {'word': 'un',
    'x': 280.0,
    'y': 125,
    'line': 0,
    'index': 1,
    'screen_word_index': 1},
   {'word': 'tiempo',
    'x': 348.0,
    'y': 125,
    'line': 0,
    'index': 2,
    'screen_word_index': 2},
   {'word': 'en',
    'x': 416.0,
    'y': 125,
    'line': 0,
    'index': 3,
    'screen_word_index': 3},
   {'word': 'que',
    'x': 463.5,
    'y': 125,
    'line': 0,
    'index': 4,
    'screen_word_index': 4},
   {'word': 'yo',
    'x': 511.0,
    'y': 125,
    'line': 0,
    'index': 5,
    'screen_word_index': 5},
   {'word': 'pensaba',
    'x': 588.5,
    'y': 125,
    'line': 0,
    'index': 6,
    'screen_word_index': 6},
   {'word': 'mucho',
    'x': 688.5,
    'y': 125,
    'line': 0,
    'index': 7,
    'screen_word_index': 7},
   {'word': 'en',
    'x': 751.0,
    'y': 125,
    'line': 0,
    'index': 8,
    'screen_

In [13]:
screens[0]["words"][1]["x"]

280.0

In [14]:
df['simulation_id'].value_counts()

simulation_id
0    2216
Name: count, dtype: int64

In [15]:
df = df[df['simulation_id'] == sim_id]

In [16]:
df[df["text_id"] == 0]

,fixation_counter,text_id,simulation_id,foveal_word,foveal_word_index,foveal_word_frequency,foveal_word_predictability,foveal_word_length,foveal_word_threshold,stimulus,...,attentional_width,saccade_type,saccade_error,saccade_distance,saccade_cause,recognition_cycle,cycle_of_recognition,fixation_duration,recognized_word_at_foveal_position,recognized_words
0,0,0,0,hubo,0,3.685921,0,4,0.5,hubo un tiempo en,...,5.0,NaN,0.000000,0.000000,0,3.0,[ 3 7 1 5 0 1 4 0 2 0 1 2 2 0 1 ...,300,hubo,['hubo' 'un' 'tiempo' 'en' 'que' 'yo' 'pensaba...
1,1,0,0,tiempo,2,4.732032,0,6,0.5,un tiempo en que yo,...,5.0,wordskip,-0.282653,7.717347,1,1.0,[ 3 7 1 5 0 1 4 0 2 0 1 2 2 0 1 ...,225,tiempo,['hubo' 'un' 'tiempo' 'en' 'que' 'yo' 'pensaba...
2,2,0,0,tiempo,2,4.732032,0,6,0.5,un tiempo en que yo,...,5.0,refixation,1.156224,2.156224,2,NaN,[ 3 7 1 5 0 1 4 0 2 0 1 2 2 0 1 ...,200,tiempo,['hubo' 'un' 'tiempo' 'en' 'que' 'yo' 'pensaba...
3,3,0,0,que,4,6.147649,0,3,0.5,en que yo pensaba mucho,...,5.0,wordskip,0.249083,6.249083,1,0.0,[ 3 7 1 5 0 1 4 0 2 0 1 2 2 0 1 ...,175,que,['hubo' 'un' 'tiempo' 'en' 'que' 'yo' 'pensaba...
4,4,0,0,pensaba,6,3.610660,0,7,0.5,yo pensaba mucho en los,...,5.0,wordskip,-0.187531,7.812469,1,4.0,[ 3 7 1 5 0 1 4 0 2 0 1 2 2 0 1 ...,250,pensaba,['hubo' 'un' 'tiempo' 'en' 'que' 'yo' 'pensaba...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,69,0,0,gris,76,2.700704,0,4,0.5,tanto gris y me acordé,...,4.0,forward,0.696199,5.696199,0,NaN,[ 3 7 1 5 0 1 4 0 2 0 1 2 2 0 1 ...,225,gris,['hubo' 'un' 'tiempo' 'en' 'que' 'yo' 'pensaba...
70,70,0,0,me,78,5.612029,0,2,0.5,y me acordé de los,...,4.5,wordskip,0.034004,5.034004,1,0.0,[ 3 7 1 5 0 1 4 0 2 0 1 2 2 0 1 ...,125,me,['hubo' 'un' 'tiempo' 'en' 'que' 'yo' 'pensaba...
71,71,0,0,acordé,79,2.176091,0,6,0.5,me acordé de los leones,...,5.0,forward,0.699930,4.699930,0,0.0,[ 3 7 1 5 0 1 4 0 2 0 1 2 2 0 1 ...,175,acordé,['hubo' 'un' 'tiempo' 'en' 'que' 'yo' 'pensaba...
72,72,0,0,los,81,5.469252,0,3,0.5,de los leones,...,5.0,wordskip,0.221198,8.221198,1,0.0,[ 3 7 1 5 0 1 4 0 2 0 1 2 2 0 1 ...,250,los,['hubo' 'un' 'tiempo' 'en' 'que' 'yo' 'pensaba...


In [17]:
def index_in_word(row):
    word = row["foveal_word"]
    stim = row["stimulus"]
    pos = row["eye_position"]
    
    # find where the word starts in the stimulus
    start = stim.index(word)
    return pos - start

df["index_in_word"] = df.apply(index_in_word, axis=1)

In [18]:
df[['foveal_word', 'stimulus', 'eye_position', 'index_in_word']]

,foveal_word,stimulus,eye_position,index_in_word
0,hubo,hubo un tiempo en,2,2
1,tiempo,un tiempo en que yo,5,2
2,tiempo,un tiempo en que yo,8,5
3,que,en que yo pensaba mucho,5,2
4,pensaba,yo pensaba mucho en los,6,3
...,...,...,...,...
2211,a,va a escribir todo esto,3,2
2212,todo,escribir todo esto sobre los,10,1
2213,esto,todo esto sobre los axolotl,7,2
2214,sobre,esto sobre los axolotl,9,4


In [19]:
# for text_id, group in df.groupby('text_id'):
#     # Reset index within group
#     group = group.reset_index(drop=True)

#     # Build the original-style DataFrame
#     output_df = pd.DataFrame()
#     output_df['index'] = range(len(group))
#     output_df['index'] = range(len(group))
#     output_df['eye'] = 'R'
#     output_df['tStart'] = group['fixation_duration'].cumsum().shift(fill_value=0).astype(int)
#     output_df['tEnd'] = output_df['tStart'] + group['fixation_duration'].astype(int)
#     output_df['duration'] = group['fixation_duration'].astype(int)
#     output_df['xAvg'] = group['eye_position']
#     output_df['yAvg'] = 150  # constant placeholder
#     output_df['pupilAvg'] = -1  # dummy placehtolder

In [20]:
CHAR_W = 15  # px per character (Courier New 18 assumption)

def idx_in_word_from_stim(stim, word, eye_pos):
    """
    Return the char index inside `word` that contains `eye_pos` in `stim`.
    Works even if the word appears multiple times.
    """
    start = stim.find(word)
    # If the word may repeat, prefer the occurrence that covers eye_pos
    if start != -1:
        # quick check first occurrence
        if start <= eye_pos < start + len(word):
            pass
        else:
            # scan all occurrences
            s = 0
            found = -1
            while True:
                s = stim.find(word, s)
                if s == -1:
                    break
                if s <= eye_pos < s + len(word):
                    found = s
                    break
                s += 1
            if found != -1:
                start = found
    # fallback clamp
    if start == -1:
        return 0
    return max(0, min(len(word) - 1, eye_pos - start))


In [21]:
eye_value = 'R'
pupil_value = -1
for i in range(df["text_id"].nunique()):
    df_screen = df[df["text_id"] == i]
    data = []
    time_acum = 0
    for idx, row in df_screen.iterrows():
        print("Palabra ob1:", row["foveal_word"])
        index = row["fixation_counter"]
        t_start = time_acum
        duration = row["fixation_duration"]
        time_acum += duration
        word_index = row["foveal_word_index"]
        dict_word = screens[i]["words"][word_index]
        word_str = row["foveal_word"]

        # compute index of the fixated character inside the word
        idx_in_word = idx_in_word_from_stim(row["stimulus"], word_str, row["eye_position"])

        # shift from the word center to the character's center
        center_x = float(dict_word["x"])
        x = center_x + (idx_in_word - (len(word_str) - 1) / 2.0) * CHAR_W

        print("Palabra dict:", dict_word)
        
        y = dict_word["y"]

        data.append({"index": index,
                     "eye": eye_value,
                     "tStart": t_start,
                     "tEnd": time_acum,
                     "duration": duration,
                     "xAvg": x,
                     "yAvg": y,
                     "pupilAvg": pupil_value              
                         })
        
    output_df = pd.DataFrame(data)
    output_dir = f"/home/lucy/Documents/facu/reading-et-main/data/processed_one/trials/DG01/Axolotl/screen_{i+1}/fixations.pkl"
    if os.path.exists(output_dir):
        os.remove(output_dir)
    output_df = pd.to_pickle(output_df, output_dir)


Palabra ob1: hubo
Palabra dict: {'word': 'Hubo', 'x': 222.5, 'y': 125, 'line': 0, 'index': 0, 'screen_word_index': 0}
Palabra ob1: tiempo
Palabra dict: {'word': 'tiempo', 'x': 348.0, 'y': 125, 'line': 0, 'index': 2, 'screen_word_index': 2}
Palabra ob1: tiempo
Palabra dict: {'word': 'tiempo', 'x': 348.0, 'y': 125, 'line': 0, 'index': 2, 'screen_word_index': 2}
Palabra ob1: que
Palabra dict: {'word': 'que', 'x': 463.5, 'y': 125, 'line': 0, 'index': 4, 'screen_word_index': 4}
Palabra ob1: pensaba
Palabra dict: {'word': 'pensaba', 'x': 588.5, 'y': 125, 'line': 0, 'index': 6, 'screen_word_index': 6}
Palabra ob1: mucho
Palabra dict: {'word': 'mucho', 'x': 688.5, 'y': 125, 'line': 0, 'index': 7, 'screen_word_index': 7}
Palabra ob1: los
Palabra dict: {'word': 'los', 'x': 797.5, 'y': 125, 'line': 0, 'index': 9, 'screen_word_index': 9}
Palabra ob1: axolotl
Palabra dict: {'word': 'axolotl.', 'x': 208.0, 'y': 175, 'line': 1, 'index': 0, 'screen_word_index': 10}
Palabra ob1: a
Palabra dict: {'word'

In [22]:
df_aver = pd.read_pickle("/home/lucy/Documents/facu/reading-et-main/data/processed_one/trials/DG01/Axolotl/screen_4/fixations.pkl")
df_aver

,index,eye,tStart,tEnd,duration,xAvg,yAvg,pupilAvg
0,0,R,0,250,250,172.5,125,-1
1,1,R,250,475,225,275.0,125,-1
2,2,R,475,675,200,352.5,125,-1
3,3,R,675,925,250,500.5,125,-1
4,4,R,925,1125,200,430.5,125,-1
...,...,...,...,...,...,...,...,...
85,85,R,21125,21375,250,465.5,575,-1
86,86,R,21375,21675,300,629.5,575,-1
87,87,R,21675,21950,275,767.5,575,-1
88,88,R,21950,22200,250,728.5,575,-1
